# LMNA: Cardiomyopathy, dilated, 1A 

The data in this workbook were derived from Table 1 of [Fatkin D, et al. (1999) Missense mutations in the rod domain of the lamin A/C gene as causes of dilated cardiomyopathy and conduction-system disease. N Engl J Med 341(23):1715-24. PMID:10580070](https://pubmed.ncbi.nlm.nih.gov/10580070/).


Variants were curated as follows
- Family A: Arg571Ser; C→A (1711): NM_005572.4(LMNA):c.1711C>A (p.Arg571Ser)
- Family B: Asn195Lys; C→G (585): NM_170707.4(LMNA):c.585C>A (p.Asn195Lys) 
- Family C: Leu85Arg; T→G (254): NM_170707.4(LMNA):c.254T>G (p.Leu85Arg) 
- Family D: Arg60Gly; C→G (178): NM_170707.4(LMNA):c.178C>G (p.Arg60Gly)
- Family E: Glu203Gly; A→G (608):  NM_170707.4(LMNA):c.608A>G (p.Glu203Gly) 

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.15


In [26]:
PMID = "PMID:10580070"
title = "Missense mutations in the rod domain of the lamin A/C gene as causes of dilated cardiomyopathy and conduction-system disease"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_excel("input/Fatkin1999.xlsx")
df.head()

,FAMILY,MEMBER,AGE AT CLINICAL PRESENTATION (YR ),Sex,CONDUCTION -SYSTEM DISEASE,ATRIAL FIBRILLATION,DILATED CARDIOMYOPATHY,CAUSE OF DEATH,COMMENTS,CK,Elevated CK,NM_005572.4
0,A,III-1,51,M,"Third-degree atrioventricular block, permanent pacemaker",No,Yes,-,NaN,181 U/liter (<210 U/liter),No,c.1711C>A
1,A,III-5,39,F,NaN,No,No,-,NaN,180 U/liter (<140 U/liter),No,c.1711C>A
2,A,III-8,42,F,"Sinus bradycardia, permanent pacemaker",Yes,Yes,-,Arterial embolus,170 U/liter (<140 U/liter),No,c.1711C>A
3,A,III-9,40,M,"Third-degree atrioventricular block, permanent pacemaker",No,Yes,Lymphoma,Heart failure,NaN,NaN,c.1711C>A
4,A,III-11,46,F,Second-degree atrioventricular block,Yes,No,-,NaN,360 U/liter (<140 U/liter),Yes,c.1711C>A


In [4]:
def create_id(row):
    return f"{row['FAMILY']}_{row['MEMBER']}"

df["patient_id"] = df.apply(create_id, axis=1)

In [5]:
ageMapper = AgeColumnMapper.by_year(column_name="AGE AT CLINICAL PRESENTATION (YR )")
#ageMapper.preview_column(df["AGE AT CLINICAL PRESENTATION (YR )"])
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(df["Sex"])

In [6]:
column_d = {}

In [7]:
df.columns

Index(['FAMILY', 'MEMBER', 'AGE AT CLINICAL PRESENTATION (YR )', 'Sex',
       'CONDUCTION -SYSTEM DISEASE', 'ATRIAL FIBRILLATION',
       'DILATED CARDIOMYOPATHY', 'CAUSE OF DEATH', 'COMMENTS', 'CK',
       'Elevated CK', 'NM_005572.4', 'patient_id'],
      dtype='object')

In [8]:
# Atrial fibrillation HP:0005110
afMapper = SimpleColumnMapper(hpo_id="HP:0005110", hpo_label="Atrial fibrillation", observed="Yes", excluded="No")
afMapper.preview_column(df['ATRIAL FIBRILLATION'])


,mapping,count
0,"original value: ""No"" -> HP: Atrial fibrillation (HP:0005110) (excluded)",16
1,"original value: ""Yes"" -> HP: Atrial fibrillation (HP:0005110) (observed)",23


In [9]:
# Dilated cardiomyopathy HP:0001644
dilatedCmMapper = SimpleColumnMapper(hpo_id="HP:0001644", hpo_label="Dilated cardiomyopathy", observed="Yes", excluded="No")
dilatedCmMapper.preview_column(df['DILATED CARDIOMYOPATHY'])

,mapping,count
0,"original value: ""Yes"" -> HP: Dilated cardiomyopathy (HP:0001644) (observed)",25
1,"original value: ""No"" -> HP: Dilated cardiomyopathy (HP:0001644) (excluded)",14


In [10]:
# Elevated circulating creatine kinase concentration HP:0003236
ckMapper = SimpleColumnMapper(hpo_id="HP:0003236", hpo_label="Elevated circulating creatine kinase concentration", observed="Yes", excluded="No")
ckMapper.preview_column(df['Elevated CK'])

,mapping,count
0,"original value: ""No"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (excluded)",20
1,"original value: ""nan"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (not measured)",18
2,"original value: ""Yes"" -> HP: Elevated circulating creatine kinase concentration (HP:0003236) (observed)",1


In [11]:
#Sudden cardiac death HP:0001645
df['CAUSE OF DEATH'].unique()
suddenMapper = SimpleColumnMapper(hpo_id="HP:0001645", hpo_label="Sudden cardiac death", observed="Sudden death", excluded="No")
suddenMapper.preview_column(df['CAUSE OF DEATH'])

,mapping,count
0,"original value: ""-"" -> HP: Sudden cardiac death (HP:0001645) (not measured)",15
1,"original value: ""Lymphoma"" -> HP: Sudden cardiac death (HP:0001645) (not measured)",1
2,"original value: ""nan"" -> HP: Sudden cardiac death (HP:0001645) (not measured)",11
3,"original value: ""Sudden death"" -> HP: Sudden cardiac death (HP:0001645) (observed)",11
4,"original value: ""No"" -> HP: Sudden cardiac death (HP:0001645) (excluded)",1


In [12]:
column_d['ATRIAL FIBRILLATION'] = afMapper
column_d['DILATED CARDIOMYOPATHY'] = dilatedCmMapper
column_d['Elevated CK'] = ckMapper
column_d['CAUSE OF DEATH'] = suddenMapper

In [13]:
df['CONDUCTION -SYSTEM DISEASE'].unique()

array(['Third-degree atrioventricular block, permanent pacemaker', nan,
       'Sinus bradycardia, permanent pacemaker',
       'Second-degree atrioventricular block',
       'Sinus bradycardia, first-degree atrioventricular block',
       'First-degree atrioventricular block',
       'Third-degree atrioventricular block', 'Yes',
       'Slow atrial fibrillation, permanent pacemaker',
       'Yes; permanent pacemaker', 'Slow atrial fibrillation', 'No',
       'Sinus bradycardia',
       'Second-degree atrioventricular block, permanent pacemaker'],
      dtype=object)

In [14]:
auto = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr, delimiter=",")

In [15]:
comments_d = {
 'Arterial embolus': 'Thromboembolism',
 'Heart failure': 'Congestive heart failure',
 'Embolic stroke': 'Stroke',
 'Left atrial enlargement': 'Left atrial enlargement',
 'Renal failure': 'Renal insufficiency',
 'Heart transplant': 'Congestive heart failure'}
commentsMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=comments_d)
commentsMapper.preview_column(df['COMMENTS'])
column_d['COMMENTS'] = commentsMapper

In [16]:
conduction_d = {'Third-degree atrioventricular block': 'Third degree atrioventricular block',
 'permanent pacemaker': 'Abnormal atrioventricular conduction',
 'Sinus bradycardia': 'Sinus bradycardia',
 'Second-degree atrioventricular block': 'Second degree atrioventricular block',
 'first-degree atrioventricular block': 'First degree atrioventricular block',
 'First-degree atrioventricular block': 'First degree atrioventricular block',
 'Yes': 'Abnormal atrioventricular conduction',
 'Slow atrial fibrillation': 'Atrial fibrillation',
 'Yes; permanent pacemaker': 'Abnormal atrioventricular conduction'
}
excluded = {'No':'Abnormal atrioventricular conduction'}
conductionMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=conduction_d)
conductionMapper.preview_column(df['CONDUCTION -SYSTEM DISEASE'])
column_d['CONDUCTION -SYSTEM DISEASE'] = conductionMapper

In [19]:
LMNA_transcript = "NM_005572.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=LMNA_transcript)
variant_d = {}
for v in df["NM_005572.4"].unique():
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var

varMapper = VariantColumnMapper(variant_d=variant_d,
                                variant_column_name='NM_005572.4', 
                                default_genotype='heterozygous')

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.1711C>A/NM_005572.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.585C>A/NM_005572.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.254T>G/NM_005572.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.178C>G/NM_005572.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.608A>G/NM_005572.4?content-type=application%2Fjson


In [22]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:115200"
omim_label = "Cardiomyopathy, dilated, 1A"
 	 
cmp1a = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=cmp1a)

In [24]:
individuals = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,6
INFORMATION,NOT_MEASURED,45


In [25]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A_III-1 (MALE; P51Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Dilated cardiomyopathy (HP:0001644); Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110); excluded: Elevated circulating creatine kinase concentration (HP:0003236)
A_III-5 (FEMALE; P39Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236)
A_III-8 (FEMALE; P42Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Dilated cardiomyopathy (HP:0001644); Thromboembolism (HP:0001907); Sinus bradycardia (HP:0001688); Abnormal atrioventricular conduction (HP:0005150); excluded: Elevated circulating creatine kinase concentration (HP:0003236)
A_III-9 (MALE; P40Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Dilated cardiomyopathy (HP:0001644); Congestive heart failure (HP:0001635); Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110)
A_III-11 (FEMALE; P46Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Elevated circulating creatine kinase concentration (HP:0003236); Second degree atrioventricular block (HP:0011706); excluded: Dilated cardiomyopathy (HP:0001644)
A_III-13 (MALE; P43Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Atrial fibrillation (HP:0005110); Sinus bradycardia (HP:0001688); First degree atrioventricular block (HP:0011705); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236)
A_III-14 (FEMALE; P40Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),First degree atrioventricular block (HP:0011705); excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644); excluded: Elevated circulating creatine kinase concentration (HP:0003236)
A_III-15 (FEMALE; P39Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.1711C>A (heterozygous),Third degree atrioventricular block (HP:0001709); excluded: Atrial fibrillation (HP:0005110); excluded: Dilated cardiomyopathy (HP:0001644)
B_III-1 (MALE; P19Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.585C>A (heterozygous),Atrial fibrillation (HP:0005110); Dilated cardiomyopathy (HP:0001644); Sudden cardiac death (HP:0001645); Congestive heart failure (HP:0001635); Abnormal atrioventricular conduction (HP:0005150)
B_III-3 (MALE; P39Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.585C>A (heterozygous),Abnormal atrioventricular conduction (HP:0005150); Dilated cardiomyopathy (HP:0001644); Sudden cardiac death (HP:0001645); Atrial fibrillation (HP:0005110)
